In [3]:
import json
import utils
from tqdm import tqdm
import traceback

Download attestation data on the block from a specific range.

In [5]:
attestation = {} # slot => attestation
for slot in tqdm(range(136000*32, 136050*32)):
    data = utils.handle_request("https://beaconcha.in/api/v1/block/{}/attestations".format(slot))
    attestation[slot] = data
with open('attestation_data.json', 'w') as fd:
    json.dump(attestation, fd, indent=4)

100%|██████████| 640/640 [20:08<00:00,  1.89s/it]  


Build data structure for each validator, including source epoch and target epoch

In [46]:
with open('attestation_data.json', 'r') as fd:
    attestation = json.load(fd)

for slot in tqdm(attestation):
    for item in attestation[slot]:
        try:
            del item['aggregationbits']
            del item['block_index']
            del item['block_root']
            del item['block_slot']
            del item['committeeindex']
            del item['signature']
            del item['source_root']
            del item['target_root']
        except Exception as e:
            print(traceback.format_exc())

validator_to_attestation = {}
for slot in tqdm(attestation):
    for item in attestation[slot]:
        try:
            for validator in item['validators']:
                record = {
                    "validator": validator,
                    "source": item['source_epoch'],
                    "target": item['target_epoch'],
                    "block_root": item['beaconblockroot'],
                    "slot": slot
                }
                if validator in validator_to_attestation:
                    validator_to_attestation[validator].append(record)
                else:
                    validator_to_attestation[validator] = [record]
        except Exception as e:
            print(traceback.format_exc())

attestation.clear()

100%|██████████| 640/640 [00:01<00:00, 404.26it/s]


Try to find unreported attestation violations in parallel

In [49]:
from concurrent import futures
import os

double_vote = []
surround_vote = []

def analyze_validators(validator):
    size = len(validator)
    for i in range(0, size):
        for j in range(i + 1, size):
            if validator[i]['block_root'] == validator[j]['block_root']:
                continue

            # Double vote: same target epoch
            if validator[i]['target'] == validator[j]['target']:
                double_vote.append({
                    'validator': validator[i]['validator'],
                    'attestation1': validator[i],
                    'attestation2': validator[j],
                })

            # Surround vote
            if validator[i]['source'] \
                < validator[j]['source'] \
                < validator[j]['target'] \
                < validator[i]['target'] or\
                validator[j]['source'] \
                < validator[i]['source'] \
                < validator[i]['target'] \
                < validator[j]['target']:
                surround_vote.append({
                    'validator': validator[i]['validator'],
                    'attestation1': validator[i],
                    'attestation2': validator[j],
                })


with futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
    results = list(tqdm(pool.map(analyze_validators, validator_to_attestation.values()), total=len(validator_to_attestation)))


100%|██████████| 76807/76807 [00:00<00:00, 195507.69it/s]


In [2]:
print(len(surround_vote), len(double_vote))

0 0
